# Student Results Scraper (Selenium + Python)

In [ ]:
# Importing the Libraries
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

results = []
regulation=int(input("enter regulation R20/R18/R14/R10"))
year=int(input("enter the year"))
branch=input("enter branch IT/CSE(CS)/ECE(EC)/EEE(EE)/CIVI(CE)/MECHANICAL(ME)/DS/CB").upper()
_type=input("enter Regular(Y)/Lateral(L)").upper()
start=int(input("enter start roll_no"))
end=int(input("enter end roll_no"))
if start<end:
    # URL
    if regulation==20:
        url = "http://becbapatla.ac.in:8080/STUDENTINFO/index_r20.html"
    elif regulation==18:
        url="http://becbapatla.ac.in:8080/STUDENTINFO/index_r18.html"
    else:
        url="http://becbapatla.ac.in:8080/STUDENTINFO/index.html"
    # Setup Selenium (use Chrome)
    driver = webdriver.Chrome()
    # Loop through roll numbers
    for roll_no in range(start, end):
        driver.get(url)
        time.sleep(1)  # let page load

        # Find the input box for roll number
        input_box = driver.find_element(By.NAME, "id")
        input_box.clear()
        input_box.send_keys(f"{_type}{year}A{branch}{roll_no}")
        input_box.send_keys(Keys.RETURN)  # To submit the 
    

        try:
            # Wait for table to appear in chrome results page
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.TAG_NAME, "table"))
            )
            # Extract Regd. No and Name
            regd_no = driver.find_element(By.XPATH, "//th[contains(text(),'Regd')]/following-sibling::td").text.strip()
            name = driver.find_element(By.XPATH, "//th[contains(text(),'Name')]/following-sibling::td").text.strip()

            # Extract Total, Percentage, CGPA by using the css indicators
            font_tag = driver.find_element(By.XPATH, "//font[@color='blue']")
    
            lines = font_tag.text.strip().split("\n")

            total = lines[0].split(":")[-1].strip()
            percentage = lines[1].split(":")[-1].strip()
            cgpa = lines[2].split(":")[-1].strip()

            results.append({
                "ROLL NO": regd_no,
                "NAME": name,
                "TOTAL": total,
                "PERCENTAGE": percentage,
                "CGPA": cgpa
            })

        except Exception: # If gets an error as noo data found after submitting the data
            # if no data found
            results.append({
                "ROLL NO": f"{_type}{year}A{branch}{roll_no}",
                "NAME": "No data found",
                "TOTAL": "No data found",
                "PERCENTAGE": "No data found",
                "CGPA": "No data found"
            })
    # Save results to CSV
    data = pd.DataFrame(results)
    data.to_csv(f"R{regulation}_{branch}_{_type}{year}_Results_.csv", index=False)
    print(f"Results saved in R{regulation}_{branch}_{_type}{year}_Results.csv")
    driver.quit()
else:
    print("please enter the roll no in order")